In [14]:
# Amend to set wd to STB_social_media_analytics.
%cd /home/jia/Desktop/git/STB_social_media_analytics/experimentation/jiaxin_experiment/MFW

/home/zyf0717/git_environment/STB_social_media_analytics


In [99]:
import pandas as pd
import mysql.connector
from scrapy import Selector
import os
import re
import yaml
import utils
from time import sleep
import traceback
from random import random
from selenium import webdriver
from datetime import datetime, date, timedelta
from random import randint

In [101]:
with open('config_file_MFW.yml') as file:
    configs = yaml.load(file, Loader=yaml.FullLoader)

chromedriver_path = configs['General']['chromedriver_path']

db_in_flag = configs['MFW']['db_in_flag']
db_out_flag = configs['MFW']['db_out_flag']

if db_in_flag == 'csv':
    
    ### FOR POC ONLY ###
    poi_index = [1, 2, 3, 4]
    poi_name = ['Gardens by the Bay', 
                'Orchard Road',
                'Empress Place Building',
                'Nagore Durgha Shrine'
               ]
    poi_url = ['https://www.mafengwo.cn/poi/5422487.html',
               'https://www.mafengwo.cn/poi/11057.html',
               'https://www.mafengwo.cn/poi/7645620.html',
               'https://www.mafengwo.cn/poi/6034067.html'
              ]
    poi_df = pd.DataFrame({'poi_index':poi_index, 
                           'poi_name':poi_name, 
                           'poi_url':poi_url}
                         )
    ####################

if db_in_flag in ['sqlite', 'mysql']:
    poi_df = pd.DataFrame()

if db_in_flag in ['sqlite', 'mysql'] or db_out_flag in ['sqlite', 'mysql']:
    cnx = mysql.connector.connect(host=configs['General']['host'],
                                  database=configs['General']['database'],
                                  user=configs['General']['user'],
                                  password=configs['General']['password']
                                 )
else:
    cnx = None

In [127]:
class CrawlMFW:
    attributes_col_names = ['POI_INDEX',
                            'TOTAL_REVIEWS',
                            'GOOD_REVIEWS',
                            'AVG_REVIEWS',
                            'BAD_REVIEWS',
                            'ABOUT',
                            'PRICE',
                            'ADDRESS',
                            'PHONE',
                            'WEBSITE',
                            'TRANSPORTATION_MODE',
                            'TIME_RECOMMENDED',
                            'OPENING_HOURS',
                            'ATTRIBUTES_CRAWLED_TIME'
                           ]

    reviews_col_names = ['REVIEW_INDEX',
                         'WEBSITE_INDEX',
                         'POI_INDEX_MFW',
                         'REVIEWER_URL',
                         'REVIEW_RATING',
                         'REVIEW_DATE',
                         'REVIEW_TIME',
                         'REVIEW_TITLE',
                         'REVIEW_BODY',
                         'TRANSLATED_REVIEW_BODY_GOOGLE',
                         'TRANSLATED_REVIEW_BODY_WATSON',
                        ]
    
    reviewers_col_names = ['REVIEWER_URL',
                           'REVIEWER_NAME',
                           'REVIEWER_LEVEL',
                           'GENDER',
                           'IMAGE_URL',
                           'CHINESE_RAW_LOCATION',
                           'CLEANED_ENGLISH_LOCATION',
                           'NUM_ATTENTIONS',
                           'NUM_FANS',
                           'REVIWER_UPDATED_TIME'
                          ]
    
    
    def __init__(self, chromedriver_path, poi_df, cnx, db_out_flag):
        self.driver = webdriver.Chrome(chromedriver_path)
        self.poi_df = poi_df
        if cnx:
            self.cursor = cnx.cursor()
        self.db_out_flag = db_out_flag
        self.number_of_pages = None
        self.current_page = None
        self.current_poi_index = None
        self.review_final_page = False
        self.attributes_df = pd.DataFrame(columns=self.attributes_col_names)
        self.reviews_df = pd.DataFrame(columns=self.reviews_col_names)
        # Create unique CSVs.
        self.datetime_string = datetime.now().strftime('%y%m%d_%H%M%S')
        os.makedirs('./output/{}/'.format(self.datetime_string))
        os.makedirs('./output/{}/reviews'.format(self.datetime_string))
        self.attributes_df.to_csv('./output/{}/attributes.csv'.format(self.datetime_string),mode='a', index=False)
                
    
    def add_to_database(self):
        # Read csv, add to database, then cnx.commit().
        return
    
    
    def crawl_pois(self, number_of_pages=None):
        if number_of_pages is not None:
            self.number_of_pages = number_of_pages
        for _, row in self.poi_df.iterrows():
            print('########## {} ##########'.format(row['poi_name']))
        # Create <POI_INDEX>.csv in reviews and reviewers folders.
            self.current_poi_index = row['poi_index']
            self.reviews_df.to_csv('./output/{}/reviews/{}.csv'.format(self.datetime_string, self.current_poi_index), mode='a', index=False)
         
        # Crawl
            try:
                self.driver.get(row['poi_url'])
                self.crawl_attributes(row['poi_index'])
                self.attributes_df.to_csv('./output/{}/attributes.csv'.format(self.datetime_string),mode='a', header=False, index=False)
                self.attributes_df = pd.DataFrame(columns=self.attributes_col_names)
                self.crawl_reviews(row['poi_index'])
                if self.db_out_flag != 'csv':
                    self.add_to_database()
            except Exception as e:
                log_file = open('./output/{}/log.txt'.format(self.datetime_string), 'a+')
                log_file.write('{}, {}, page: {}, {}\n'.format(row['poi_index'], row['poi_name'], self.current_page, datetime.now()))
                # log_file.write(repr(e) + '\n')
                log_file.write(traceback.format_exc() + '\n')
                log_file.close()
                break
    
    def crawl_reviews(self, poi_index):
        if self.number_of_pages is not None:
            self.current_page = 1
            for i in range(self.number_of_pages):
                print('Page {}'.format(self.current_page))
                self.crawl_reviews_1_page(poi_index)
                self.reviews_to_csv()
                self.current_page += 1
                if self.review_final_page:
                    self.review_final_page= False
                    break
        else:
            self.current_page = 1
            while not self.review_final_page:
                print('Page {}'.format(self.current_page))
                self.crawl_reviews_1_page(poi_index)
                self.reviews_to_csv()
                self.current_page += 1
                if self.review_final_page:
                    self.review_final_page= False
                    break
    
    
    def crawl_attributes(self, poi_index):
        driver = self.driver
        
        # Crawling attributes elements.
        #Forming selector path
    sleep(randint(1,5)) 
    sel = Selector(text=driver.page_source)
    sleep(randint(1,5)) 

    res = sel.xpath('.//div[@class="container"]')   

    xpath_name='//div[@class="row row-top"]//div[@class="title"]/h1/text()'   #passed

    xpath_eng='//div[@class="row row-top"]//div[@class="en"]/text()'    
    eng = res.xpath(xpath_eng).extract_first() 

    xpath_total_reviews = '//div[@class="mhd mhd-large"]/span/em/text()'    
    total_reviews = res.xpath(xpath_total_reviews).extract_first() 

    xpath_good_reviews = '//li[@data-category="13"]/a/span[@class="num"]/text()'   
    good_reviews = res.xpath(xpath_good_reviews).extract_first()

    xpath_avg_reviews = '//li[@data-category="12"]/a/span[@class="num"]/text()'   
    avg_reviews = res.xpath(xpath_avg_reviews).extract_first()

    xpath_bad_reviews = '//li[@data-category="11"]/a/span[@class="num"]/text()'  
    bad_reviews = res.xpath(xpath_bad_reviews).extract_first()
    
    #special case, there are many different special tags
    #even worse, may share same xpath
    xpath_special_reviews = '//li[@data-category="1"]/a/span[@class="num"]/text()'    
    special_reviews = res.xpath(xpath_special_reviews).extract_first()

    xpath_about1='//div[@data-cs-p="概况"]/div/text()'  #specific location   
    xpath_about2='//div[@class="main-detail"]/dl[1]/dd/text()'   #general location
    if res.xpath(xpath_about2).extract_first() != None:
        about = (driver.find_element_by_xpath('//div[@class="main-detail"]/dl[1]/dd')).text
    else:
        about = (driver.find_element_by_xpath('//div[@data-cs-p="概况"]/div')).text

    #xpath_price = the price shown here is a list and may not directly related to the POI(may be a travel package)
 

    xpath_address1 ='//div[@class="mhd"]/p/text()'   #景点位置   
    xpath_address2 ='//div[@class="address"]/text()'   #地址     
    if res.xpath(xpath_address1).extract_first() != None:
        address = res.xpath(xpath_address1).extract_first()
    else:
        address = res.xpath(xpath_address2).extract_first()


    xpath_phone1 ='//li[@class="tel"]/div[@class="content"]/text()'   #specific location   
    xpath_phone2 ='//li[@class="item-tel"]/div[@class="content"]/text()'   #general   
    if res.xpath(xpath_phone1).extract_first() != None:   
        phone = res.xpath(xpath_phone1).extract_first()
    else:
        phone = res.xpath(xpath_phone2).extract_first()                        


    xpath_website = '//li[@class="item-site"]/div[@class="content"]/a/@href'  
    website = res.xpath(xpath_website).extract_first()     


    xpath_rec_time ='//li[@class="item-time"]/div[@class="content"]/text()'   #only appeared for specific location, but share sme xpath
    if res.xpath(xpath_address1).extract_first() != None:    #confirm its a specific location
        rec_time = res.xpath(xpath_rec_time).extract_first()
    else:
        rec_time = 'NA'
    
    xpath_op_hr1 = '//div[@class="mod mod-detail"]/dl[3]/dd/text()'     #specific location case 1
    xpath_op_hr1_1 = '//div[@class="mod mod-detail"]/dl[3]/dd//div/text()'   #specific location case 2
    xpath_op_hr2 ='//li[@class="item-time"]/div[@class="content"]/text()'
    if res.xpath(xpath_address1).extract_first() != None:   #confirm it is a specific location
        if res.xpath(xpath_op_hr1).extract_first() != None:
            op_hr=res.xpath(xpath_op_hr1).extract_first()
        else:
            op_hr=res.xpath(xpath_op_hr1_1).extract_first()
    else:
        op_hr=res.xpath(xpath_op_hr2).extract_first()  #general
    
    
    xpath_trans1 = '//div[@class="mod mod-detail"]/dl[1]/dd/text()'    #specific location   
    xpath_trans2 = '//div[@class="main-detail"]/dl[3]/dd/text()'    #general   
    if res.xpath(xpath_trans1).extract_first() != None:
        transportation = res.xpath(xpath_trans1).extract_first()
    else:
        transportation = res.xpath(xpath_trans2).extract_first()

        
        # Parsing attributes.
        
        
        poi_attributes = [poi_index,
                          total_reviews, 
                          ranking, 
                          average_rating, 
                          rating_breakdown[0],
                          rating_breakdown[1],
                          rating_breakdown[2],
                          rating_breakdown[3],
                          rating_breakdown[4],
                          about, 
                          address, 
                          datetime.now()
                         ]
        
        # Inserting attributes into dataframe
        poi_attributes_dict = dict(zip(self.attributes_col_names, poi_attributes))
        self.attributes_df = self.attributes_df.append(poi_attributes_dict, ignore_index=True)

    def crawl_reviews_1_page(self, poi_index, earliest_date=None):        
        driver = self.driver
        
        # If crawl all languages, uncomment the follwing 3 lines.
        # all_languages_button = driver.find_element_by_xpath('//span[@class="location-review-review-list-parts-LanguageFilter__no_wrap--2Dckv"]')
        # all_languages_button.click()
        # sleep(1)
        
        read_more_button = driver.find_element_by_xpath('//span[@class="location-review-review-list-parts-ExpandableReview__cta--2mR2g"]')
        read_more_button.click()
        sleep(1)

        # Crawling review elements.
        reviewer_url_elements = driver.find_elements_by_xpath('//a[@class="ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC"]')
        reviewer_details_elements = driver.find_elements_by_xpath('//div[@class="social-member-event-MemberEventOnObjectBlock__event_wrap--1YkeG"]')
        review_id_elements = driver.find_elements_by_xpath('//div[@class="location-review-review-list-parts-SingleReview__mainCol--1hApa"]')
        review_rating_elements = driver.find_elements_by_xpath('//div[@class="location-review-review-list-parts-RatingLine__bubbles--GcJvM"]/span')
        review_title_elements = driver.find_elements_by_xpath('//a[@class="location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT"]')
        review_body_elements = driver.find_elements_by_xpath('//div[@class="location-review-review-list-parts-ExpandableReview__containerStyles--1G0AE"]')
        date_of_experience_elements = driver.find_elements_by_xpath('//span[@class="location-review-review-list-parts-EventDate__event_date--1epHa"]')
        
        for i in range(len(reviewer_url_elements)):
            
            # Parsing review and reviewer details
            reviewer_url = reviewer_url_elements[i].get_attribute('href')
            reviewer_name = reviewer_url_elements[i].text
            review_id = self.parse_review_id_elements(review_id_elements[i].get_attribute('data-reviewid'))
            review_date = self.parse_review_date(reviewer_details_elements[i].text)
            location_contribution_votes = self.parse_location_contributions_votes(reviewer_details_elements[i].text)
            review_rating = self.parse_review_rating(review_rating_elements[i].get_attribute('class'))
            review_title = review_title_elements[i].text
            review_body = self.parse_review_body(review_body_elements[i].text)
            date_of_experience = self.parse_date_of_experience(review_body_elements[i].text)
            trip_type = self.parse_trip_type(review_body_elements[i].text)
            
            review_details = [None, # REVIEW_INDEX
                              1, # WEBSITE_INDEX (TripAdvisor is '1')
                              poi_index,
                              reviewer_url,
                              review_id,
                              review_date,
                              review_rating,
                              review_title,
                              review_body,
                              date_of_experience,
                              trip_type,
                              datetime.now()
                             ]
            
            reviewer_details = [reviewer_url,
                                reviewer_name,
                                location_contribution_votes[0],
                                None, # CLEANED_HOME_LOCATION
                                location_contribution_votes[1],
                                location_contribution_votes[2],
                                datetime.now()
                               ]
            
            # Inserting reviews into dataframe.
            review_details_dict = dict(zip(self.reviews_col_names, review_details))
            self.reviews_df = self.reviews_df.append(review_details_dict, ignore_index=True)
            
            # Inserting reviewers into dataframe.
            reviewer_details_dict = dict(zip(self.reviewers_col_names, reviewer_details))
            self.reviewers_df = self.reviewers_df.append(reviewer_details_dict, ignore_index=True)

        next_button = driver.find_element_by_xpath('//a[@class="ui_button nav next primary "]')
        if next_button != []:
            next_button.click()
            sleep(1)
    
    # Methods below are all utility functions.
    def calculate_total_reviews(self, rating_breakdown):
        return sum(rating_breakdown)
    
    
    def parse_ranking_text(self, text):
        return int(text[1:text.find(' of')].replace(',', ''))
    
    
    def calculate_average_rating(self, rating_breakdown):
        total = sum(rating_breakdown)
        average = 0
        for i, j in enumerate(rating_breakdown[::-1]):
            average += (i+1)*j/total
        return average
    
    
    def parse_rating_breakdown_elements(self, elements):
        rating_breakdown = []
        for element in elements:
            text = element.text
            rating_breakdown.append(int(text.replace(",", "")))
        return rating_breakdown
    
    
    def parse_address_text(self, text_1, text_2, text_3, text_4):
        return ('{}, {}, {} {}'.format(text_1, text_2, text_3, text_4))
    
    
    def parse_review_date(self, text):
        date_string = text[text.find('wrote a review ')+15:text.find('\n')]
        
        if date_string == 'Today':
            return datetime.now().strftime('%d-%m-%Y')
        elif date_string == 'Yesterday':
            return (datetime.now() - timedelta(1)).strftime('%d-%m-%Y')
        
        re_search = re.search('(\d+) (\w+)', date_string)
        current_year = datetime.now().strftime('%Y')
        if re_search is not None:
            if len(re_search.group(1)) == 1:
                return datetime.strptime('0' + date_string + ' ' + current_year, '%d %b %Y').strftime('%d-%m-%Y')
            else:
                return datetime.strptime(date_string + ' ' + current_year, '%d %b %Y').strftime('%d-%m-%Y')
        
        return datetime.strptime(date_string, '%b %Y').strftime('%m-%Y')                     
    
    
    def parse_location_contributions_votes(self, text):
        location, contributions, votes = None, None, None
        
        votes_search = re.search('(\d+) helpful votes?', text)
        if votes_search is not None:
            votes = int(votes_search.group(1))
                
        contributions_search = re.search('(\d+) contributions?', text)
        if contributions_search is not None:
            contributions = int(contributions_search.group(1))

        location_search = re.search('(.+?){} contributions?'.format(contributions), text)
        if location_search is not None:
            location = location_search.group(1)

        return location, contributions, votes
    
    
    def parse_review_id_elements(self, text):
        return int(text)
    
    
    def parse_review_rating(self, text):
        return int(text[-2:])//10
    
    
    def parse_review_body(self, text):
        return text[:text.find('Read less')-1]
    
    
    def parse_date_of_experience(self, text):
        substring = re.search('Date of experience: (.+)\n', text).group(1)
        return datetime.strptime(substring, '%B %Y').strftime('%m-%Y')  
    
    
    def parse_trip_type(self, text):
        if text.find('Trip type: ') == -1:
            return None
        substring = text[text.find('Trip type: ')+11:]
        return substring[:substring.find('\n')]
    

In [128]:
CrawlTripAdvisor(chromedriver_path, poi_df, cnx, db_out_flag).crawl_pois(number_of_pages=2)

In [98]:
datetime.now().strftime('%y%m%d_%H%M%S')

'200116_152740'

In [103]:
'12 Jan'.split()

['12', 'Jan']